# Get true color imagery for water quality data from sentinel 3 ocli

Author: J. Hester

6/25/2021

EO Dashboard Hackathon

In [36]:
from edc import check_compatibility
#check_compatibility("user-0.24.5", dependencies=["SH"])
import s3fs
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import xcube.core.store
from xcube.core.dsio import open_cube
import nc_time_axis
plt.rcParams["figure.figsize"] = 14,8
import imageio
import os
import pandas as pd
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
#from requests_oauthlib import requests
import geojson
import shapely.wkt
import shapely.geometry
import IPython.display
from sentinelhub import (MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, 
                         DataCollection, bbox_to_dimensions, DownloadRequest, SHConfig)
from osgeo import gdal
from PIL import Image

### Configuration

In [37]:
config = SHConfig()
config.sh_client_id = os.environ["SH_CLIENT_ID"]
config.sh_client_secret = os.environ["SH_CLIENT_SECRET"]

In [42]:
# Define the plot_image function
# param save false to not save iamges or a save name and path
# param title the iamge title (embedded inside image)
def plot_image(image, factor=1.0, clip_range = None, save=False, title='', **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    
    extent = [coords_wgs84[0],coords_wgs84[2], coords_wgs84[1], coords_wgs84[3]] # puts the coords in the right place on the output plot
    
    if clip_range is not None:
        #ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
        ax.imshow(np.clip(image * factor, *clip_range), origin='upper', extent=extent, interpolation='none')
        ax.title.set_text(title)
        if save:
            ax.figure.savefig(save)
    else:
        #ax.imshow(image * factor, **kwargs)
        ax.imshow(image * factor, origin='lower', extent=extent, interpolation='none')
        ax.title.set_text(title)
        if save:
            ax.figure.savefig(save)
    ax.set_xticks([])
    ax.set_yticks([])

### User-specified values

In [43]:
#set rectangular study area by using coordinates at opposite corners
# example is of ise bay
coords_wgs84 = [137.01690785604998,34.51149653877783, 136.46484486776873, 35.100097358716525]
region_name = 'Ise Bay' # the name of the area your rectangle surrounds to be used in image

In [44]:
resolution = 60 # m resolution size to get
betsiboka_bbox = BBox(bbox=coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

In [45]:
start_images = "2020-01-01" # first day to get images
end_images = "2020-06-01" # last day to get images
image_output_dir = "tc_images" # where to save the images

### Generate water quality imagery

In [46]:
# param save_name the name of the output file, including the extension
# param img_title the name of the image (such as location/date) to embed inside
def generate_tc_tiffs(save_name, img_title, request_true_color):
    color_imgs = request_true_color.get_data()

    image_c = color_imgs[0]
    plot_image(image_c, factor=3.5/255, clip_range=(0,1), save=save_name, title=img_title)

B02 = Yellow substance and detrital pigments (turbidity)

B04 = Chlorophyll

B05 = Chlorophyll, sediment, turbidity, red tide

Change these values accordingly in evalscript_wq below. See https://docs.sentinel-hub.com/api/latest/data/sentinel-3-olci-l1b/ for more

In [47]:
# param start the start date to get images
# param end the end date
# param output_dir the folder to save the images in
def time_lapse_tc_images(start, end, output_dir):
    global start_date, end_date, evalscript_wq, request_chlorophyll
    dates = pd.date_range(start, end).tolist()
    for date in dates:
        d = str(date).split(' ')[0]
        start_date, end_date = d, d
        
        evalscript_true_color = """
            //VERSION=3

            function setup() {
                return {
                    input: [{
                        bands: ["B02", "B03", "B04"]
                    }],
                    output: {
                        bands: 3
                    }
                };
            }

            function evaluatePixel(sample) {
                return [sample.B04, sample.B03, sample.B02];
            }
        """

        request_true_color = SentinelHubRequest(
            evalscript=evalscript_true_color,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L1C,
                    time_interval=(start_date, end_date),
                )
            ],
            responses=[
                SentinelHubRequest.output_response('default', MimeType.PNG)
            ],
            bbox=betsiboka_bbox,
            size=betsiboka_size,
            config=config
        )
        
        # update these as relevant to the location/save location
        save_name = output_dir+'/tc_map'+d+'.png'
        img_title =  region_name + ' ' + d# This needs to be updated for the location
        generate_tc_tiffs(save_name, img_title, request_true_color)

In [48]:
time_lapse_tc_images(start_images, end_images,image_output_dir)